In [9]:
# install all required PIP packages
!pip install -r requirements.txt

# load all required libraries
import re
import unittest
import openai
import ipywidgets
import pyperclip
import textwrap
import ipywidgets
from IPython.display import display, clear_output

In [3]:
# Display interactive UI for generating prompts.

# Define the event handler for the generate button click
def on_generate_button_click(button):
    # Get the input text and call the generate_prompt function
    generate_prompt(text_input.value)

# Define the event handler for the clear button click
def on_clear_button_click(button):
    # Clear the output of the cell
    clear_output(wait=True)
    # Redisplay the widgets
    display(label, text_input, generate_button, clear_button)

# Create the label, text input, and button widgets
label = ipywidgets.Label("Description of image for prompt:")
text_input = ipywidgets.Text()
generate_button = ipywidgets.Button(description="Generate")
clear_button = ipywidgets.Button(description="New Prompt")

# Set the event handlers for the button clicks
generate_button.on_click(on_generate_button_click)
clear_button.on_click(on_clear_button_click)

# Display the widgets
display(label, text_input, generate_button, clear_button)



Label(value='Description of image for prompt:')

Text(value='')

Button(description='Generate', style=ButtonStyle())

Button(description='New Prompt', style=ButtonStyle())

Image description:
A flaming rainbow pirate ship.



NameError: name 'system_prompt' is not defined

Image description:
A flaming rainbow pirate ship.

Generated Midjourney prompt:
A pirate ship with a flaming rainbow sail and hull sailing on choppy seas.
Elements: The ship has a jolly roger flag with a skull and crossbones. The ship
is surrounded by seagulls. The captain has a red bandana and a gold hoop
earring. The ship has cannons on the deck. There is a treasure chest on the
deck. --chaos 66 --no signature

Processed prompt:
A pirate ship with a flaming rainbow sail and hull sailing on choppy seas. The
ship has a jolly roger flag with a skull and crossbones. The ship is surrounded
by seagulls. The captain has a red bandana and a gold hoop earring. The ship has
cannons on the deck. There is a treasure chest on the deck. --chaos 66 --no
signature



In [4]:
# OpenAI API setup.

# Fetch OpenAI API key from file.
with open('openai_api_key.txt', 'r') as f:
    openai.api_key = f.read().strip()

In [10]:
# Read the system prompt from system_prompt.txt

# Define the default base prompt and user input prompt
with open('system_prompt.txt', 'r') as f:
    system_prompt = f.read().strip()

# Print the combined prompt, wrapped
print("System prompt:")
print(textwrap.fill(system_prompt, width=80))

System prompt:
You are a prompt generator for the Midjourney AI image generation service.  Each
of my requests will be a request for you to generate a prompt for the Midjourney
AI image generation service.  Start each prompt with a sentence that starts with
a noun phrase in proper English that describes the medium of the image in
detail.  For example, "A black-and-white street photograph of a...", or "An
impressionist oil painting of a...", or "A children's book illustration of
a...", etc.  The full sentence should start with that noun phrase for the
medium, and it should end with a noun phrase that describes the image in detail.
The medium and the image should be compatible.  For example, if the user prompt
is "a person smiling", then the medium should not be "a landscape painting",
because a landscape painting is not a medium for a person smiling.  When you
write the sentence, add details to the description of the image that the user
did not provide in their prompt. For example, if t

In [5]:
# Define the function to be called when the button is clicked.

def generate_prompt(description):
    print("Image description:")
    print(description + "\n")

    response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        temperature = 0.8,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": description},
        ])

    assistant_reply = response['choices'][0]['message']['content'].strip()

    print("Generated Midjourney prompt:")
    print(textwrap.fill(assistant_reply, width=80) + "\n")

    print("Processed prompt:")
    processed_prompt = process_prompt(assistant_reply)
    print(textwrap.fill(processed_prompt, width=80) + "\n")

    # Copy the prompt to the clipboard.
    pyperclip.copy(processed_prompt)

# Process the generated prompt to remove things that cause problems for Midjourney.
def process_prompt(prompt):
    # Remove stray hyphens surrounded by whitespace
    prompt = re.sub(r'\s-\s', ' ', prompt)
    
    # Remove hyphens at the beginning of words but not from parameters
    prompt = re.sub(r'(?<=\s)-(?=[^-])', '', prompt)

    # Remove "Elements:" and "Midjourney parameters:" phrases
    prompt = re.sub(r'(Elements:|Midjourney parameters:)', '', prompt)
    
    # Remove invalid parameters.
    valid_params = ["chaos", "aspect", "quality", "style", "no"]
    prompt = remove_invalid_parameters(prompt, valid_params)

    return prompt


In [7]:
# Remove parameters that GPT-3 generates but are not valid for Midjourney.

def remove_invalid_parameters(s, allowed_params):
    s = re.sub(r'--(\w+)\s+(\S+)', lambda m: f"--{m.group(1)} {m.group(2)}" if m.group(1) in allowed_params else "", s)
    s = re.sub(r'\s+', ' ', s)
    return s.strip()

class TestRemoveInvalidParameters(unittest.TestCase):
    maxDiff = None

    def test_remove_invalid_parameters(self):
        input_text = '''\
A sepia-toned vintage postcard from Miami in the 1920s, featuring a bustling
beach scene with palm trees and parasols.   Elements: People in bathing suits,
wooden boardwalk, old-fashioned beach umbrellas, a distant pier, vintage cars
parked along the beach.   --aspect 3:2 --chaos 66 --q 2 --no signature
'''

        expected_output = '''\
A sepia-toned vintage postcard from Miami in the 1920s, featuring a bustling
beach scene with palm trees and parasols. Elements: People in bathing suits,
wooden boardwalk, old-fashioned beach umbrellas, a distant pier, vintage cars
parked along the beach. --aspect 3:2 --chaos 66 --no signature
'''
        expected_output = ' '.join(expected_output.splitlines())

        allowed_params = ['chaos', 'aspect', 'style', 'quality', 'no']
        cleaned_s = remove_invalid_parameters(input_text, allowed_params)

        self.assertEqual(cleaned_s, expected_output)

# Run the test case
unittest.main(argv=[''], exit=False)



.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
